In [1]:
!pip install datasets

In [2]:
# Import TensorFlow and TensorFlow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import seaborn as sns

# Import ResNet50 model and layers from Keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from sklearn.metrics import confusion_matrix

# Import Matplotlib for plotting
import matplotlib.pyplot as plt

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the datasets
# Define input shape for ResNet50
input_shape = (224, 224, 3)

# Preprocess the validation dataset
def preprocess(image, label):
    image = tf.image.resize(image, (input_shape[0], input_shape[1]))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [4]:
# Define the preprocessing function with data augmentation
def preprocess_with_augmentation(image, label):
    # Apply random transformations to the image
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.7, upper=1.3)

    # Resize image to match ResNet50 input size
    image = tf.image.resize(image, (input_shape[0], input_shape[1]))

    # Normalize pixel values
    image = tf.cast(image, tf.float32) / 255.0

    return image, label

(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    'malaria',
    split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],
    as_supervised=True,
    with_info=True,
)

# Apply data augmentation to the training dataset
ds_train_augmented = ds_train.map(preprocess_with_augmentation)

# Batch and prefetch the augmented dataset
ds_train_augmented  = ds_train_augmented.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

test_ds = ds_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

ds_validation = ds_validation.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
# Define the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output) # Global average pooling layer
x = Dense(256, activation='relu')(x) # Fully connected layer with ReLU activation
x = Dropout(0.8)(x)
output = Dense(1, activation='sigmoid')(x) # Output layer with sigmoid activation (binary classification)
model = Model(inputs=base_model.input, outputs=output)


94765736/94765736 [==============================] - 3s 0us/step


In [6]:
# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model on the augmented training dataset, validating on the validation dataset
history = model.fit(ds_train_augmented, epochs=9, validation_data=ds_validation)

Epoch 1/9
603/603 [==============================] - 2810s 5s/step - loss: 0.1498 - accuracy: 0.9500 - val_loss: 1.4275 - val_accuracy: 0.5035
Epoch 2/9
603/603 [==============================] - 2622s 4s/step - loss: 0.1058 - accuracy: 0.9632 - val_loss: 0.1071 - val_accuracy: 0.9644
Epoch 3/9
603/603 [==============================] - 2616s 4s/step - loss: 0.0956 - accuracy: 0.9680 - val_loss: 0.0850 - val_accuracy: 0.9710
Epoch 4/9
603/603 [==============================] - 2560s 4s/step - loss: 0.0876 - accuracy: 0.9702 - val_loss: 0.1013 - val_accuracy: 0.9615
Epoch 5/9
603/603 [==============================] - 2624s 4s/step - loss: 0.0886 - accuracy: 0.9710 - val_loss: 0.0952 - val_accuracy: 0.9644
Epoch 6/9
300/603 [=============>................] - ETA: 23:00 - loss: 0.0805 - accuracy: 0.9731

In [ ]:
# Evaluate the model on the validation dataset
evaluation = model.evaluate(ds_validation)

loss = evaluation[0]
accuracy = evaluation[1]

# Print validation loss and accuracy
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy*100:.2f}%, Test Loss: {test_loss:.4f}")

In [ ]:
y_pred = model.predict(test_ds)
y_pred_binary = (y_pred > 0.5).astype(int)
y_test = np.concatenate([y for x, y in test_ds], axis=0)

cnf_mat = confusion_matrix(y_test, y_pred_binary)

#Code to print confusion matrix as a heatmap
plt.figure(figsize=(10,8))
sns.heatmap(cnf_mat, annot=True, fmt="d")
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
model.save('malaria_model3.keras')
print("Model exported successfully.")